## MongoDB Atlas Quickstart

[MongoDB Atlas Vector Search](https://www.mongodb.com/products/platform/atlas-vector-search) is part of the MongoDB platform that enables MongoDB customers to build intelligent applications powered by semantic search over any type of data. Atlas Vector Search allows you to integrate your operational database and vector search in a single, unified, fully managed platform with full vector database capabilities.

You can integrate TruLens with your application built on Atlas Vector Search to leverage observability and measure improvements in your application's search capabilities.

This tutorial will walk you through the process of setting up TruLens with MongoDB Atlas Vector Search and Llama-Index as the orchestrator.

Even better, you'll learn how to use metadata filters to create specialized query engines and leverage a router to choose the most appropriate query engine based on the query.

See [MongoDB Atlas/LlamaIndex Quickstart](https://www.mongodb.com/docs/atlas/atlas-vector-search/ai-integrations/llamaindex/) for more details.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/expositional/vector-dbs/mongodb_atlas/atlas_quickstart.ipynb)



In [ ]:
# !pip install trulens-eval llama-index llama-index-vector-stores-mongodb llama-index-embeddings-openai pymongo

## Import TruLens and start the dashboard

In [1]:
from trulens_eval import Tru

tru = Tru()

tru.reset_database()

tru.run_dashboard()

/Users/jreini/opt/anaconda3/envs/trulens_dev_empty/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.
Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.4.23:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Set imports, keys and llama-index settings

In [2]:
import getpass, os, pymongo, pprint
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.core.settings import Settings
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.vector_stores import MetadataFilter, MetadataFilters, ExactMatchFilter, FilterOperator
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch

In [3]:
import os
os.environ["OPENAI_API_KEY"] = "sk-..."
ATLAS_CONNECTION_STRING = "mongodb+srv://<username>:<password>@<clusterName>.<hostname>.mongodb.net"

In [4]:
Settings.llm = OpenAI()
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")
Settings.chunk_size = 100
Settings.chunk_overlap = 10

## Load sample data

Here we'll load two PDFs: one for Atlas best practices and one textbook on database essentials.

In [5]:
# Load the sample data
!mkdir -p 'data/'
!wget 'https://query.prod.cms.rt.microsoft.com/cms/api/am/binary/RE4HkJP' -O 'data/atlas_best_practices.pdf'
atlas_best_practices = SimpleDirectoryReader(input_files=["./data/atlas_best_practices.pdf"]).load_data()

!wget 'http://fondamentidibasididati.it/wp-content/uploads/2020/11/DBEssential-2021-C30-11-21.pdf' -O 'data/DBEssential-2021.pdf'
db_essentials = SimpleDirectoryReader(input_files=["./data/DBEssential-2021.pdf"]).load_data()

documents = atlas_best_practices + db_essentials


--2024-04-05 16:06:56--  https://query.prod.cms.rt.microsoft.com/cms/api/am/binary/RE4HkJP
Resolving query.prod.cms.rt.microsoft.com (query.prod.cms.rt.microsoft.com)... 23.47.30.189
Connecting to query.prod.cms.rt.microsoft.com (query.prod.cms.rt.microsoft.com)|23.47.30.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/pdf]
Saving to: ‘data/atlas_best_practices.pdf’

data/atlas_best_pra     [ <=>                ] 500.64K  --.-KB/s    in 0.04s   

2024-04-05 16:06:56 (11.2 MB/s) - ‘data/atlas_best_practices.pdf’ saved [512653]

--2024-04-05 16:06:56--  http://fondamentidibasididati.it/wp-content/uploads/2020/11/DBEssential-2021-C30-11-21.pdf
Resolving fondamentidibasididati.it (fondamentidibasididati.it)... 157.138.8.2
Connecting to fondamentidibasididati.it (fondamentidibasididati.it)|157.138.8.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 595602 (582K) [application/pdf]
Saving to: ‘data/DBEssential-2021

## Create a vector store

Next you need to create an Atlas Vector Search Index.

When you do so, use the following in the json editor:

```
{
  "fields": [
    {
      "numDimensions": 1536,
      "path": "embedding",
      "similarity": "cosine",
      "type": "vector"
    },
    {
      "path": "metadata.file_name",
      "type": "filter"
    }
  ]
}
```

In [6]:
# Connect to your Atlas cluster
mongodb_client = pymongo.MongoClient(ATLAS_CONNECTION_STRING)

# Instantiate the vector store
atlas_vector_search = MongoDBAtlasVectorSearch(
    mongodb_client,
    db_name = "atlas-best-practices",
    collection_name = "test",
    index_name = "vector_index"
)
vector_store_context = StorageContext.from_defaults(vector_store=atlas_vector_search)

# load both documents into the vector store
vector_store_index = VectorStoreIndex.from_documents(
   documents, storage_context=vector_store_context, show_progress=True
)


Parsing nodes:   0%|          | 0/133 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1144 [00:00<?, ?it/s]

## Setup basic RAG

In [7]:
query_engine = vector_store_index.as_query_engine()

## Add feedback functions

In [8]:
from trulens_eval.feedback.provider import OpenAI
from trulens_eval import Feedback
import numpy as np

# Initialize provider class
provider = OpenAI()

# select context to be used in feedback. the location of context is app specific.
from trulens_eval.app import App
context = App.select_context(query_engine)

from trulens_eval.feedback import Groundedness
grounded = Groundedness(groundedness_provider=OpenAI())
# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(context.collect()) # collect context chunks into a list
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name = "Answer Relevance")
    .on_input_output()
)
# Context relevance between question and each context chunk.
f_context_relevance = (
    Feedback(provider.context_relevance_with_cot_reasons, name = "Context Relevance")
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.query.rets.source_nodes[:].node.text .


[nltk_data] Downloading package punkt to /Users/jreini/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
from trulens_eval import TruLlama
tru_query_engine_recorder = TruLlama(query_engine,
    app_id='Basic RAG',
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance])

## Write test cases

Let's write a few test queries to test the ability of our RAG to answer questions on both documents in the vector store.

In [14]:
from trulens_eval.generate_test_set import GenerateTestSet

test_set = {'MongoDB Atlas': [
                "How do you secure MongoDB Atlas?",
                "How can Time to Live (TTL) be used to expire data in MongoDB Atlas?"],
            'Database Essentials': [
                "Why might I use a relational data model?",
                "What is the impact of interleaving transactions in database operations?"]
}

## Get testing!

Our test set is made up of 2 topics (test breadth), each with 2 questions (test depth).

We can store the topic as record level metadata and then test queries from each topic, using `tru_query_engine_recorder` as a context manager.

In [15]:
with tru_query_engine_recorder as recording:
    for category in test_set:
        recording.record_metadata=dict(prompt_category=category)
        test_prompts = test_set[category]
        for test_prompt in test_prompts:
            response = query_engine.query(test_prompt)

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

## Check evaluation results

Evaluation results can be viewed in the TruLens dashboard (started at the top of the notebook) or directly in the notebook.

In [16]:
tru.get_leaderboard()

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
Basic RAG,0.625,0.775,0.85,1.25,0.000388


Perhaps if we use metadata filters to create specialized query engines, we can improve the search results and thus, the overall evaluation results.

But it may be clunky to have two separate query engines - then we have to decide which one to use!

Instead, let's use a router query engine to choose the query engine based on the query.

## Router Query Engine + Metadata Filters

In [20]:
# Specify metadata filters
metadata_filters_db_essentials = MetadataFilters(
   filters=[ExactMatchFilter(key="metadata.file_name", value="DBEssential-2021.pdf")]
)
metadata_filters_atlas = MetadataFilters(
   filters=[ExactMatchFilter(key="metadata.file_name", value="atlas_best_practices.pdf")]
)

# Instantiate Atlas Vector Search as a retriever for each set of filters
vector_store_retriever_db_essentials = VectorIndexRetriever(index=vector_store_index, filters=metadata_filters_db_essentials, similarity_top_k=5)
vector_store_retriever_atlas = VectorIndexRetriever(index=vector_store_index, filters=metadata_filters_atlas, similarity_top_k=5)

# Pass the retrievers into the query engines
query_engine_with_filters_db_essentials = RetrieverQueryEngine(retriever=vector_store_retriever_db_essentials)
query_engine_with_filters_atlas = RetrieverQueryEngine(retriever=vector_store_retriever_atlas)

from llama_index.core.tools import QueryEngineTool

# Set up the two distinct tools (query engines)

essentials_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine_with_filters_db_essentials,
    description=(
        "Useful for retrieving context about database essentials"
    ),
)

atlas_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine_with_filters_atlas,
    description=(
        "Useful for retrieving context about MongoDB Atlas"
    ),
)

# Create the router query engine

from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector, LLMMultiSelector
from llama_index.core.selectors import (
    PydanticMultiSelector,
    PydanticSingleSelector,
)


router_query_engine = RouterQueryEngine(
    selector=PydanticSingleSelector.from_defaults(),
    query_engine_tools=[
        essentials_tool,
        atlas_tool,
    ],
)

from trulens_eval import TruLlama
tru_query_engine_recorder_with_router = TruLlama(router_query_engine,
    app_id='Router Query Engine + Filters',
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance])

In [21]:
with tru_query_engine_recorder_with_router as recording:
    for category in test_set:
        recording.record_metadata=dict(prompt_category=category)
        test_prompts = test_set[category]
        for test_prompt in test_prompts:
            response = router_query_engine.query(test_prompt)

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/2 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/1 [00:00<?, ?it/s]

## Check results!

In [22]:
tru.get_leaderboard()

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
Router Query Engine + Filters,0.725,0.950,0.779167,1.25,0.000926
Basic RAG,0.625,0.775,0.850000,1.25,0.000388
